In [1]:
import numpy as np
import pandas as pd
import os

In [9]:
input_folder = 'input'
example_file = 'seq_0.fasta.ascii_pssm'

In [15]:
def to_dataframes(pssm_file):

    file_content = open(pssm_file, 'r').readlines()
    
    aas_header = ["idx", "AA", *" ".join(file_content[2][:-1].split()).split(' '), "information per position", " relative weight of gapless real matches to pseudocounts"]
    order_dict = {}
    for i, aa in enumerate(aas_header):
        if aa not in order_dict.values():
            order_dict[i] = aa
        else:
            order_dict[i] = aa+"_pc"

    columns = order_dict.values()
    pssm_matrix_df = pd.DataFrame(columns=columns)
    
    for line in file_content[3:]:

        if line == "\n":
            break

        content = " ".join(line[:-1].split()).split(' ')
        content_dict = {}
        for i, value in enumerate(content):
            content_dict[order_dict[i]] = value
        pssm_matrix_df = pd.concat([pssm_matrix_df, pd.DataFrame([content_dict], columns=columns)])
        
    return pssm_matrix_df

In [37]:
def to_encoding(pssm_file):
    file_content = open(pssm_file, 'r').readlines()
    values_list = []
    for line in file_content[3:]:
        if line == "\n":
            break
        values_list.append(np.array(" ".join(line[:-1].split()).split(' ')[2:22]))
        
    return np.array(values_list)

In [43]:
def to_encoding_pd(pssm_file):
    pssm_df = to_dataframes(pssm_file)
    selected_columns = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
    # Save values from selected columns in an array
    pssm_array = pssm_df[selected_columns].values
    return pssm_array

In [44]:
def folder_to_encoding(pssm_folder):
    # For each file in pssm_folder, parse it and save it in a list
    pssm_list = []
    for pssm_file in os.listdir(pssm_folder):
        pssm_array = to_encoding(os.path.join(pssm_folder, pssm_file))
        pssm_list.append(pssm_array)
    return np.array(pssm_list)

In [45]:
folder_to_encoding("/home/jabarbero/MERGE/jba/data/BRCA1_HUMAN_Fields2015_y2h/blast_fastas/blast_results/pssm_results").shape

(1278, 303, 20)